# Simulation Alpha Range Tester
This notebook allows us to iterate over different values of $\mu_a$ for different simulation raw files at test the resulting range of alphas. The final goal is to determine the required simulation space(range) for $\mu_a$ such that we get results comparable to TFO PPG data. This notebook is not automatic. Rather it serves to insert values for $mu_a$'s and instantly observe their effect. 

In [2]:
from glob import glob
from pathlib import Path
import  numpy as np
from inverse_modelling_tfo.tools.name_decoder import decode_extended_filename
from inverse_modelling_tfo.tools.intensity_data_generator import intensity_from_raw
from inverse_modelling_tfo.data.intensity_interpolation import interpolate_exp_chunk

selected_wave_int = 1
selected_uterus_thickness = 5
selected_maternal_wall_thickness = 12

raw_data_path = Path('/home/rraiyan/simulations/tfo_sim/data/raw_dan_iccps_equispace_detector')
all_files = glob(str(raw_data_path.joinpath('*.pkl')))

selected_file = None
for file in all_files:
    if (selected_maternal_wall_thickness, selected_uterus_thickness, selected_wave_int) == decode_extended_filename(file):
        selected_file = file
        break
if selected_file is None:
    print("File Not Found")

In [3]:
# MU MAP = 1 : Maternal Wall, 2: Uterus, 3: Amniotic Fluid, 4: Fetal Tissue
test_maternal_mu_a = 0.005
mu_map_base1 = {1: 0.0091, 2: 0.0158, 3: 0.0125, 4: 0.013}  # 735nm
mu_map_base2 = {1: 0.0087, 2: 0.0991, 3: 0.042, 4: 0.012}   # 850nm
mu_map_active = mu_map_base1 if selected_wave_int == 1 else mu_map_base2

In [4]:
processed_data = intensity_from_raw(selected_file, mu_map_active, 1.0)

In [5]:
sdd = processed_data['SDD'].to_numpy()
processed_data['Intensity'] /= 1e9
processed_data['Intensity'] /= np.pi * 2 ** 2
detector_count = [11, 16, 22, 27, 32, 38, 43, 48, 53,
                  59, 64, 69, 75, 80, 85, 90, 96, 101, 106, 111]
sdd_to_detector_count_map = {
    dist: count for dist, count in zip(sdd, detector_count)}
processed_data['Intensity'] /= processed_data['SDD'].map(sdd_to_detector_count_map)

print(processed_data)

    SDD     Intensity
0    10  1.422993e-04
1    14  2.300250e-05
2    19  4.283837e-06
3    23  1.112823e-06
4    28  3.201815e-07
5    32  9.814937e-08
6    37  3.084679e-08
7    41  1.084131e-08
8    46  3.690279e-09
9    50  1.387119e-09
10   55  5.501001e-10
11   59  2.301662e-10
12   64  1.044989e-10
13   68  5.074476e-11
14   73  2.614608e-11
15   77  1.328429e-11
16   82  7.411706e-12
17   86  4.063591e-12
18   91  2.114381e-12
19   95  1.312687e-12


In [11]:
alphas = interpolate_exp_chunk(processed_data, [1, 1], return_alpha=True)
# print(alphas.flatten())
print(alphas)

[[-12.62756712]
 [  0.22873078]
 [-12.9125372 ]
 [ 19.58686133]]
